モデルを少し変えたい

In [1]:
import warnings, random, os, sys, tqdm, time
sys.path.append("../")
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, RobustScaler

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import KFold

import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.modules.loss import _WeightedLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau

from pytorch_tabnet.tab_model import TabNetRegressor
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

pd.set_option("display.max_columns", 1200)
pd.set_option("display.max_rows", 1200)
%matplotlib inline



In [2]:
def metric(y_true, y_pred):
    res = []
    for i in range(0, y_true.shape[1]):
        y = y_true[:,i]
        pred = y_pred[:,i]
        print(i)
        res.append(log_loss(y, pred))
    return np.mean(res)

In [3]:
def metric(y_true, y_pred):
    res = []
    for i in range(0, y_true.shape[1]):
        y = y_true[:,i]
        pred = y_pred[:,i]
        if np.sum(pred) <= 0.0:
            pre += 1e-15
        res.append(log_loss(y, pred))
    return np.mean(res)

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True
seed_everything(42)
        
    
def make_scaler(flag, seed):
    if flag == "quantile":
        return QuantileTransformer(n_quantiles=100,random_state=seed, output_distribution="normal")
    elif flag == "gauss":
        return GaussRankScaler()
    elif flag == "standard":
        return StandardScaler()
    elif flag == "minmax":
        return MinMaxScaler()
    elif flag == "robust":
        return RobustScaler()
    
seeds = [7, 8, 9, 10, 11, 12, 13]
SCALE = "quantile"
    


In [4]:
# g772, c100, 206クラス、402クラスの分類

train_df = pd.read_csv("../../../Data/Raw/train_features.csv")
test_df = pd.read_csv("../../../Data/Raw/test_features.csv")
#pub_test_df = pd.read_csv("../input/moapublictest/test_features.csv")
pub_test_df = pd.read_csv("../../../Data/Raw/test_features.csv")
drug_df = pd.read_csv("../../../Data/Raw/train_drug.csv")#

y = pd.read_csv("../../../Data/Raw/train_targets_scored.csv")
y_non = pd.read_csv("../../../Data/Raw/train_targets_nonscored.csv")
y_all = pd.concat([y, y_non.drop("sig_id", axis=1)], axis=1)
y = y.merge(drug_df, on='sig_id', how='left') #

GENES = [col for col in train_df.columns if col.startswith("g-")]
CELLS = [col for col in train_df.columns if col.startswith("c-")]
BIOS = GENES + CELLS


SCORED_MOAS = [col for col in y.columns if col != "sig_id" and col != "drug_id"]#
NONSCORED_MOAS = [col for col in y_non.columns if col != "sig_id"]
ALL_MOAS = SCORED_MOAS + NONSCORED_MOAS


TR_SIZE = train_df.shape[0]
TE_SIZE = test_df.shape[0]

train_nonvehicle_index = train_df[train_df["cp_type"] != "ctl_vehicle"].index
test_nonvehicle_index = test_df[test_df["cp_type"] != "ctl_vehicle"].index

train_df["time_dose"] = train_df["cp_time"].astype(str) + " * " + train_df["cp_dose"]
test_df["time_dose"] = test_df["cp_time"].astype(str) + " * " + test_df["cp_dose"]
pub_test_df["time_dose"] = pub_test_df["cp_time"].astype(str) + " * " + pub_test_df["cp_dose"]

# remove cp_type = ctl_vehicle
mask = train_df["cp_type"] != "ctl_vehicle"
train_df = train_df[mask].drop("cp_type", axis=1).reset_index(drop=True)
test_df = test_df[test_df["cp_type"] != "ctl_vehicle"].drop("cp_type", axis=1).reset_index(drop=True)
pub_test_df = pub_test_df[pub_test_df["cp_type"] != "ctl_vehicle"].drop("cp_type", axis=1).reset_index(drop=True)
y_nonv = y[mask].reset_index(drop=True)#

scored = y_nonv.copy()#
y_nonv.drop("drug_id", axis=1, inplace=True)#
y.drop("drug_id", axis=1, inplace=True)#

TR_NONV_SIZE = train_df.shape[0]
TE_NONV_SHAPE = test_df.shape[0]

In [5]:
"""# prod
#prod_p_cols = [['g-722', 'g-655', 'g-707'], ['g-707', 'g-65', 'g-392'], ['g-169', 'g-484', 'g-338'], ['g-417', 'g-100', 'g-707'], ['g-38', 'g-100', 'g-707'], ['g-310', 'g-744', 'g-707'], ['g-100', 'g-0', 'g-123'], ['g-38', 'g-100', 'g-744'], ['g-328', 'g-707', 'g-158'], ['g-100', 'g-744', 'g-38'], ['g-310', 'g-744', 'g-707'], ['g-491', 'g-100', 'g-38'], ['g-135', 'g-392', 'g-512'], ['g-131', 'g-38', 'g-708'], ['g-180', 'g-624', 'g-613'], ['g-707', 'g-133', 'g-392'], ['g-69', 'g-707', 'g-100'], ['g-392', 'g-731', 'g-707'], ['g-759', 'g-392', 'g-65'], ['g-544', 'g-425', 'g-707'], ['g-69', 'g-608', 'g-417'], ['g-441', 'g-703', 'g-491'], ['g-712', 'g-310', 'g-328'], ['g-624', 'g-615', 'g-189'], ['g-57', 'g-729', 'g-130'], ['g-146', 'g-466', 'g-762'], ['g-308', 'g-495', 'g-712'], ['g-181', 'g-707', 'g-38'], ['g-392', 'g-731', 'g-131'], ['g-349', 'g-750', 'g-91'], ['g-541', 'g-748', 'g-38'], ['g-91', 'g-100', 'g-478'], ['g-635', 'g-514', 'g-302'], ['g-419', 'g-676', 'g-130'], ['g-744', 'g-131', 'g-100'], ['g-707', 'g-158', 'g-100'], ['g-127', 'g-749', 'g-380'], ['g-392', 'g-731', 'g-100'], ['g-144', 'g-123', 'g-86'], ['g-732', 'g-744', 'g-707'], ['g-744', 'g-731', 'g-100'], ['g-731', 'g-158', 'g-38'], ['g-158', 'g-100', 'g-707'], ['g-208', 'g-707', 'g-731'], ['g-38', 'g-392', 'g-707'], ['g-744', 'g-721', 'g-707'], ['g-162', 'g-157', 'g-178'], ['g-326', 'g-707', 'g-449'], ['g-504', 'g-392', 'g-707'], ['g-729', 'g-182', 'g-208'], ['g-744', 'g-608', 'g-100'], ['g-452', 'g-391', 'g-413'], ['g-714', 'g-452', 'g-658'], ['g-100', 'g-392', 'g-707'], ['g-640', 'g-266', 'g-310'], ['g-91', 'g-145', 'g-208'], ['g-744', 'g-158', 'g-392'], ['g-16', 'g-714', 'g-707'], ['g-310', 'g-13', 'g-100'], ['g-478', 'g-468', 'g-310'], ['g-689', 'g-100', 'g-707'], ['g-208', 'g-714', 'g-707'], ['g-38', 'g-158', 'g-707'], ['g-484', 'g-392', 'g-544'], ['g-392', 'g-484', 'g-74'], ['g-95', 'g-170', 'g-707'], ['g-91', 'g-130', 'g-707'], ['g-131', 'g-208', 'g-392'], ['g-417', 'g-248', 'g-744'], ['g-707', 'g-607', 'g-358'], ['g-392', 'g-707', 'g-158'], ['g-31', 'g-328', 'g-460'], ['g-576', 'g-666', 'g-608'], ['g-368', 'g-402', 'g-707'], ['g-512', 'g-594', 'g-38'], ['g-38', 'g-707', 'g-158'], ['g-392', 'g-100', 'g-707'], ['g-91', 'g-100', 'g-707'], ['g-158', 'g-38', 'g-744'], ['g-744', 'g-707', 'g-100'], ['g-654', 'g-143', 'g-377'], ['g-131', 'g-100', 'g-170'], ['g-699', 'g-235', 'g-707'], ['g-744', 'g-392', 'g-38'], ['g-682', 'g-592', 'g-707'], ['g-391', 'g-666', 'g-514'], ['g-143', 'g-231', 'g-265'], ['g-478', 'g-442', 'g-270'], ['g-608', 'g-162', 'g-11'], ['g-134', 'g-54', 'g-762'], ['g-145', 'g-201', 'g-208'], ['g-413', 'g-310', 'g-707'], ['g-744', 'g-38', 'g-100'], ['g-413', 'g-707', 'g-69'], ['g-123', 'g-731', 'g-100'], ['g-712', 'g-208', 'g-38'], ['g-731', 'g-707', 'g-100'], ['g-38', 'g-744', 'g-100'], ['g-576', 'g-452', 'g-392'], ['g-441', 'g-157', 'g-392'], ['g-596', 'g-744', 'g-707'], ['g-38', 'g-392', 'g-744'], ['g-392', 'g-69', 'g-654'], ['g-123', 'g-744', 'g-38'], ['g-417', 'g-46', 'g-181'], ['g-731', 'g-100', 'g-707'], ['g-484', 'g-707', 'g-158'], ['g-744', 'g-707', 'g-100'], ['g-100', 'g-158', 'g-707'], ['g-200', 'g-707', 'g-592'], ['g-215', 'g-392', 'g-330'], ['g-383', 'g-576', 'g-514'], ['g-689', 'g-69', 'g-100'], ['g-645', 'g-123', 'g-514'], ['g-697', 'g-200', 'g-608'], ['g-91', 'g-100', 'g-392'], ['g-38', 'g-707', 'g-158'], ['g-231', 'g-100', 'g-707'], ['g-573', 'g-127', 'g-70'], ['g-178', 'g-592', 'g-391'], ['g-368', 'g-100', 'g-707'], ['g-200', 'g-729', 'g-648'], ['g-723', 'g-731', 'g-100'], ['g-130', 'g-745', 'g-158'], ['g-208', 'g-131', 'g-100'], ['g-392', 'g-38', 'g-91'], ['g-731', 'g-744', 'g-158'], ['g-437', 'g-158', 'g-91'], ['g-707', 'g-335', 'g-116'], ['g-100', 'g-731', 'g-707'], ['g-44', 'g-714', 'g-707'], ['g-392', 'g-413', 'g-707'], ['g-65', 'g-392', 'g-158'], ['g-592', 'g-157', 'g-197'], ['g-740', 'g-762', 'g-170'], ['g-137', 'g-188', 'g-181'], ['g-270', 'g-170', 'g-100'], ['g-260', 'g-190', 'g-130'], ['g-718', 'g-722', 'g-707'], ['g-146', 'g-270', 'g-744'], ['g-505', 'g-391', 'g-38'], ['g-100', 'g-731', 'g-707'], ['g-84', 'g-65', 'g-707'], ['g-65', 'g-484', 'g-707'], ['g-328', 'g-28', 'g-392'], ['g-624', 'g-608', 'g-707'], ['g-635', 'g-158', 'g-707'], ['g-419', 'g-143', 'g-640'], ['g-744', 'g-707', 'g-100'], ['g-158', 'g-707', 'g-12'], ['g-744', 'g-707', 'g-38'], ['g-666', 'g-466', 'g-191'], ['g-38', 'g-328', 'g-417'], ['g-328', 'g-38', 'g-100'], ['g-417', 'g-392', 'g-707'], ['g-130', 'g-608', 'g-707'], ['g-380', 'g-38', 'g-707'], ['g-707', 'g-391', 'g-624'], ['g-613', 'g-330', 'g-106'], ['g-131', 'g-100', 'g-744'], ['g-181', 'g-512', 'g-270'], ['g-181', 'g-100', 'g-392'], ['g-744', 'g-417', 'g-100'], ['g-158', 'g-208', 'g-159'], ['g-88', 'g-643', 'g-436'], ['g-484', 'g-158', 'g-707'], ['g-100', 'g-744', 'g-392'], ['g-188', 'g-689', 'g-290'], ['g-158', 'g-417', 'g-11'], ['g-328', 'g-220', 'g-541'], ['g-243', 'g-259', 'g-744'], ['g-392', 'g-313', 'g-422'], ['g-707', 'g-131', 'g-123'], ['g-208', 'g-100', 'g-707'], ['g-158', 'g-417', 'g-707'], ['g-392', 'g-313', 'g-389'], ['g-16', 'g-707', 'g-100'], ['g-231', 'g-707', 'g-392'], ['g-725', 'g-712', 'g-640'], ['g-38', 'g-158', 'g-707'], ['g-201', 'g-745', 'g-599'], ['g-419', 'g-158', 'g-714'], ['g-173', 'g-596', 'g-737'], ['g-326', 'g-248', 'g-600'], ['g-131', 'g-100', 'g-158'], ['g-654', 'g-160', 'g-707'], ['g-147', 'g-731', 'g-721'], ['g-378', 'g-358', 'g-643'], ['g-100', 'g-38', 'g-744'], ['g-166', 'g-635', 'g-430'], ['g-417', 'g-38', 'g-328'], ['g-208', 'g-100', 'g-707'], ['g-38', 'g-744', 'g-100'], ['g-229', 'g-711', 'g-91'], ['g-248', 'g-38', 'g-338'], ['g-38', 'g-417', 'g-707'], ['g-731', 'g-100', 'g-707'], ['g-106', 'g-744', 'g-91'], ['g-707', 'g-624', 'g-455'], ['g-310', 'g-707', 'g-290'], ['g-131', 'g-170', 'g-158'], ['g-146', 'g-270', 'g-158'], ['g-529', 'g-707', 'g-74']]
#prod_n_cols = [['g-431', 'g-597', 'g-489'], ['g-239', 'g-113', 'g-50'], ['g-370', 'g-431', 'g-656'], ['g-568', 'g-508', 'g-760'], ['g-370', 'g-508', 'g-37'], ['g-228', 'g-72', 'g-67'], ['g-50', 'g-37', 'g-250'], ['g-508', 'g-50', 'g-411'], ['g-128', 'g-370', 'g-429'], ['g-50', 'g-672', 'g-37'], ['g-508', 'g-37', 'g-489'], ['g-50', 'g-705', 'g-298'], ['g-771'], ['g-67', 'g-644', 'g-113'], ['g-50', 'g-37', 'g-36'], ['g-151', 'g-495', 'g-234'], ['g-276', 'g-178', 'g-428'], ['g-644', 'g-370', 'g-411'], ['g-370', 'g-568', 'g-50'], ['g-50', 'g-37', 'g-332'], ['g-375', 'g-644', 'g-271'], ['g-571', 'g-503', 'g-370'], ['g-56', 'g-161', 'g-298'], ['g-508', 'g-37', 'g-370'], ['g-537', 'g-487', 'g-719'], ['g-211', 'g-75', 'g-501'], ['g-503', 'g-477', 'g-489'], ['g-508', 'g-113', 'g-231'], ['g-113', 'g-375', 'g-75'], ['g-50', 'g-332', 'g-37'], ['g-113', 'g-75', 'g-178'], ['g-644', 'g-178', 'g-760'], ['g-50', 'g-72', 'g-37'], ['g-653', 'g-202', 'g-378'], ['g-300', 'g-247', 'g-584'], ['g-37', 'g-50', 'g-98'], ['g-50', 'g-58', 'g-332'], ['g-411', 'g-674', 'g-299'], ['g-672', 'g-50', 'g-508'], ['g-128', 'g-370', 'g-644'], ['g-37', 'g-228', 'g-202'], ['g-508', 'g-760', 'g-406'], ['g-75', 'g-113', 'g-228'], ['g-508', 'g-50', 'g-370'], ['g-72', 'g-370', 'g-50'], ['g-370', 'g-75', 'g-37'], ['g-399', 'g-644', 'g-739'], ['g-674', 'g-555', 'g-713'], ['g-610', 'g-537', 'g-323'], ['g-276', 'g-113', 'g-555'], ['g-421', 'g-612', 'g-464'], ['g-370', 'g-50', 'g-503'], ['g-508', 'g-151', 'g-267'], ['g-258', 'g-644', 'g-370'], ['g-370', 'g-276', 'g-75'], ['g-50', 'g-560', 'g-630'], ['g-332', 'g-178', 'g-508'], ['g-50', 'g-37', 'g-75'], ['g-370', 'g-558', 'g-371'], ['g-37', 'g-62', 'g-271'], ['g-75', 'g-50', 'g-375'], ['g-37', 'g-421', 'g-72'], ['g-370', 'g-684', 'g-508'], ['g-50', 'g-508', 'g-113'], ['g-50', 'g-298', 'g-37'], ['g-239', 'g-370', 'g-739'], ['g-705', 'g-50', 'g-298'], ['g-37', 'g-5', 'g-332'], ['g-508', 'g-37', 'g-72'], ['g-513', 'g-508', 'g-128'], ['g-555', 'g-281', 'g-172'], ['g-50', 'g-37', 'g-489'], ['g-370', 'g-228', 'g-644'], ['g-370', 'g-719', 'g-508'], ['g-298', 'g-477', 'g-644'], ['g-257', 'g-50', 'g-72'], ['g-75', 'g-370', 'g-672'], ['g-508', 'g-113', 'g-370'], ['g-332', 'g-58', 'g-37'], ['g-508', 'g-37', 'g-672'], ['g-50', 'g-37', 'g-672'], ['g-681', 'g-272', 'g-131'], ['g-771'], ['g-50', 'g-37', 'g-672'], ['g-370', 'g-571', 'g-438'], ['g-508', 'g-332', 'g-50'], ['g-370', 'g-400', 'g-300'], ['g-300', 'g-284', 'g-495'], ['g-234', 'g-761', 'g-555'], ['g-257', 'g-672', 'g-477'], ['g-370', 'g-227', 'g-653'], ['g-238', 'g-399', 'g-759'], ['g-508', 'g-228', 'g-656'], ['g-370', 'g-618', 'g-644'], ['g-508', 'g-228', 'g-113'], ['g-67', 'g-571', 'g-653'], ['g-37', 'g-228', 'g-489'], ['g-67', 'g-178', 'g-113'], ['g-202', 'g-370', 'g-421'], ['g-50', 'g-113', 'g-672'], ['g-370', 'g-399', 'g-98'], ['g-401', 'g-58', 'g-202'], ['g-508', 'g-370', 'g-58'], ['g-75', 'g-37', 'g-50'], ['g-300', 'g-370', 'g-568'], ['g-508', 'g-37', 'g-50'], ['g-234', 'g-271', 'g-632'], ['g-67', 'g-760', 'g-50'], ['g-113', 'g-75', 'g-489'], ['g-37', 'g-50', 'g-508'], ['g-508', 'g-67', 'g-37'], ['g-555', 'g-492', 'g-653'], ['g-113', 'g-250', 'g-325'], ['g-705', 'g-178', 'g-284'], ['g-332', 'g-50', 'g-571'], ['g-508', 'g-67', 'g-276'], ['g-375', 'g-644', 'g-113'], ['g-298', 'g-257', 'g-332'], ['g-75', 'g-50', 'g-653'], ['g-204', 'g-558', 'g-202'], ['g-587', 'g-644', 'g-17'], ['g-204', 'g-567', 'g-526'], ['g-571', 'g-568', 'g-760'], ['g-653', 'g-555', 'g-678'], ['g-202', 'g-50', 'g-760'], ['g-50', 'g-298', 'g-332'], ['g-50', 'g-37', 'g-59'], ['g-67', 'g-113', 'g-508'], ['g-75', 'g-298', 'g-508'], ['g-508', 'g-50', 'g-298'], ['g-487', 'g-285', 'g-232'], ['g-370', 'g-644', 'g-50'], ['g-370', 'g-575', 'g-477'], ['g-50', 'g-257', 'g-37'], ['g-508', 'g-644', 'g-113'], ['g-513', 'g-591', 'g-300'], ['g-375', 'g-681', 'g-492'], ['g-571', 'g-234', 'g-412'], ['g-37', 'g-58', 'g-228'], ['g-17', 'g-115', 'g-300'], ['g-323', 'g-506', 'g-168'], ['g-771'], ['g-743', 'g-497', 'g-595'], ['g-375', 'g-370', 'g-508'], ['g-50', 'g-37', 'g-568'], ['g-50', 'g-438', 'g-439'], ['g-370', 'g-67', 'g-58'], ['g-477', 'g-568', 'g-761'], ['g-370', 'g-267', 'g-705'], ['g-234', 'g-58', 'g-520'], ['g-234', 'g-508', 'g-595'], ['g-37', 'g-50', 'g-202'], ['g-37', 'g-644', 'g-489'], ['g-477', 'g-72', 'g-37'], ['g-506', 'g-719', 'g-300'], ['g-370', 'g-508', 'g-37'], ['g-508', 'g-50', 'g-37'], ['g-370', 'g-50', 'g-113'], ['g-370', 'g-300', 'g-487'], ['g-568', 'g-276', 'g-719'], ['g-421', 'g-434', 'g-644'], ['g-595', 'g-176', 'g-487'], ['g-50', 'g-672', 'g-489'], ['g-508', 'g-50', 'g-250'], ['g-250', 'g-489', 'g-439'], ['g-72', 'g-644', 'g-406'], ['g-185', 'g-37', 'g-672'], ['g-300', 'g-227', 'g-591'], ['g-332', 'g-37', 'g-370'], ['g-50', 'g-37', 'g-489'], ['g-367', 'g-438', 'g-144'], ['g-37', 'g-50', 'g-257'], ['g-276', 'g-653', 'g-291'], ['g-50', 'g-761', 'g-672'], ['g-477', 'g-571', 'g-585'], ['g-370', 'g-113', 'g-508'], ['g-508', 'g-370', 'g-332'], ['g-508', 'g-75', 'g-113'], ['g-58', 'g-186', 'g-477'], ['g-202', 'g-479', 'g-660'], ['g-477', 'g-332', 'g-36'], ['g-239', 'g-42', 'g-234'], ['g-508', 'g-370', 'g-37'], ['g-37', 'g-300', 'g-370'], ['g-370', 'g-300', 'g-96'], ['g-5', 'g-429', 'g-299'], ['g-477', 'g-497', 'g-487'], ['g-152', 'g-50', 'g-75'], ['g-375', 'g-477', 'g-585'], ['g-228', 'g-411', 'g-67'], ['g-190', 'g-475', 'g-628'], ['g-202', 'g-37', 'g-477'], ['g-595', 'g-276', 'g-75'], ['g-72', 'g-75', 'g-508'], ['g-370', 'g-508', 'g-50'], ['g-37', 'g-75', 'g-121'], ['g-503', 'g-761', 'g-50'], ['g-595', 'g-486', 'g-72'], ['g-508', 'g-67', 'g-370'], ['g-50', 'g-508', 'g-75'], ['g-508', 'g-553', 'g-72'], ['g-595', 'g-665', 'g-131'], ['g-705', 'g-375', 'g-704'], ['g-75', 'g-228', 'g-508'], ['g-508', 'g-37', 'g-644'], ['g-477', 'g-50', 'g-72']]
# 上位500こ
prod_cols = [['g-145', 'g-201', 'g-208'], ['g-370', 'g-508', 'g-37'], ['g-38', 'g-392', 'g-707'], ['g-328', 'g-28', 'g-392'], ['g-441', 'g-157', 'g-392'], ['g-181', 'g-100', 'g-392'], ['g-67', 'g-760', 'g-50'], ['g-731', 'g-100', 'g-707'], ['g-478', 'g-468', 'g-310'], ['g-91', 'g-145', 'g-208'], ['g-106', 'g-744', 'g-91'], ['g-131', 'g-208', 'g-392'], ['g-144', 'g-123', 'g-86'], ['g-228', 'g-72', 'g-67'], ['g-31', 'g-328', 'g-460'], ['g-392', 'g-731', 'g-100'], ['g-732', 'g-744', 'g-707'], ['g-705', 'g-375', 'g-704'], ['g-508', 'g-50', 'g-411'], ['g-234', 'g-58', 'g-520'], ['g-503', 'g-761', 'g-50'], ['g-113', 'g-75', 'g-178'], ['g-50', 'g-508', 'g-113'], ['g-113', 'g-375', 'g-75'], ['g-576', 'g-452', 'g-392'], ['g-50', 'g-37', 'g-36'], ['g-707', 'g-133', 'g-392'], ['g-484', 'g-392', 'g-544'], ['g-508', 'g-67', 'g-370'], ['g-123', 'g-731', 'g-100'], ['g-298', 'g-477', 'g-644'], ['g-72', 'g-370', 'g-50'], ['g-67', 'g-178', 'g-113'], ['g-744', 'g-608', 'g-100'], ['g-91', 'g-100', 'g-707'], ['g-37', 'g-228', 'g-202'], ['g-37', 'g-300', 'g-370'], ['g-234', 'g-508', 'g-595'], ['g-596', 'g-744', 'g-707'], ['g-300', 'g-227', 'g-591'], ['g-135', 'g-392', 'g-512'], ['g-731', 'g-744', 'g-158'], ['g-69', 'g-707', 'g-100'], ['g-276', 'g-653', 'g-291'], ['g-624', 'g-615', 'g-189'], ['g-181', 'g-707', 'g-38'], ['g-72', 'g-75', 'g-508'], ['g-231', 'g-707', 'g-392'], ['g-508', 'g-37', 'g-72'], ['g-725', 'g-712', 'g-640'], ['g-67', 'g-644', 'g-113'], ['g-508', 'g-228', 'g-656'], ['g-185', 'g-37', 'g-672'], ['g-370', 'g-50', 'g-503'], ['g-201', 'g-745', 'g-599'], ['g-332', 'g-50', 'g-571'], ['g-50', 'g-37', 'g-59'], ['g-508', 'g-113', 'g-231'], ['g-707', 'g-158', 'g-100'], ['g-257', 'g-50', 'g-72']]
    
for cols in prod_cols:
    name = "prod-" + " * ".join(cols)
    train_df[name] = train_df[cols].mean(axis=1)
    test_df[name] = test_df[cols].mean(axis=1)
    pub_test_df[name] = pub_test_df[cols].mean(axis=1)"""

PRODS = [col for col in train_df.columns if col.startswith("prod-")]

In [6]:
#out fold preprocessing

#variance threshold

VAR_THRESHOLD = 0.8
drop_cols = []
temp = pd.concat([train_df, pub_test_df])
for col in BIOS+PRODS:
    if temp[col].var() <= VAR_THRESHOLD:
        drop_cols.append(col)

print("drop cols num : {}".format(len(drop_cols)))
train_df.drop(columns=drop_cols, inplace=True)
test_df.drop(columns=drop_cols, inplace=True)
pub_test_df.drop(columns=drop_cols, inplace=True)

GENES_ = [col for col in train_df.columns if col.startswith("g-")]
CELLS_ = [col for col in train_df.columns if col.startswith("c-")]
BIOS_ = GENES_ + CELLS_
        
del temp

# onehot encode of categorical feature and drop
drop_cols = ["cp_time", "cp_dose", "time_dose"]
train_df = pd.concat([pd.get_dummies(train_df["time_dose"], prefix="onehot", drop_first=True), train_df.drop(drop_cols, axis=1) ], axis=1)
test_df = pd.concat([pd.get_dummies(test_df["time_dose"], prefix="onehot", drop_first=True), test_df.drop(drop_cols, axis=1) ], axis=1)
pub_test_df = pd.concat([pd.get_dummies(pub_test_df["time_dose"], prefix="onehot", drop_first=True), pub_test_df.drop(drop_cols, axis=1) ], axis=1)

# aggregation feature
print("agg")
for df in [train_df, pub_test_df, test_df]:
    df["sum-g"] = df[GENES_].sum(axis=1)
    df["mean-g"] = df[GENES_].mean(axis=1)
    df["std-g"] = df[GENES_].std(axis=1)
    df["kurt-g"] = df[GENES_].kurt(axis=1)
    df["skew-g"] = df[GENES_].skew(axis=1)
    df["sum-c"] = df[CELLS_].sum(axis=1)
    df["mean-c"] = df[CELLS_].mean(axis=1)
    df["std-c"] = df[CELLS_].std(axis=1)
    df["kurt-c"] = df[CELLS_].kurt(axis=1)
    df["skew-c"] = df[CELLS_].skew(axis=1)
    df["sum-gc"] = df[BIOS_].sum(axis=1)
    df["mean-gc"] = df[BIOS_].mean(axis=1)
    df["std-gc"] = df[BIOS_].std(axis=1)
    df["kurt-gc"] = df[BIOS_].kurt(axis=1)
    df["skew-gc"] = df[BIOS_].skew(axis=1)


drop cols num : 67
agg


In [7]:
X = train_df.drop("sig_id", axis=1)
y_nonv = y_nonv.drop("sig_id", axis=1).values
y = y.drop("sig_id", axis=1).values

## Dateset Class

In [8]:
class MoAResNetDataset:
    def __init__(self, features1, features2, targets):
        self.features1 = features1
        self.features2 = features2
        self.targets = targets
        
    def __len__(self):
        return (self.features1.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x1' : torch.tensor(self.features1[idx, :], dtype=torch.float),
            'x2' : torch.tensor(self.features2[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestResNetDataset:
    def __init__(self, features1, features2):
        self.features1 = features1
        self.features2 = features2
        
    def __len__(self):
        return (self.features1.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x1' : torch.tensor(self.features1[idx, :], dtype=torch.float),
            'x2' : torch.tensor(self.features2[idx, :], dtype=torch.float)
        }
        return dct

## func 

In [9]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    for data in dataloader:
        optimizer.zero_grad()
        inputs1, inputs2, targets = data['x1'].to(device), data['x2'].to(device), data['y'].to(device)
        outputs = model(inputs1, inputs2)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        # if cycle
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    
    for data in dataloader:
        inputs1, inputs2, targets = data['x1'].to(device), data['x2'].to(device), data['y'].to(device)
        outputs = model(inputs1, inputs2)
        loss = loss_fn(outputs, targets)
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    for data in dataloader:
        inputs1, inputs2 = data['x1'].to(device), data['x2'].to(device)

        with torch.no_grad():
            outputs = model(inputs1, inputs2)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

## Model

In [10]:
"""class Model(nn.Module):
    def __init__(self, num_features1, num_features2, num_targets):
        super(Model, self).__init__()
        self.h1_1 = 512
        self.h1_2 = 256
        
        self.h2_1 = num_features2+self.h1_2
        self.h2_2 = 512
        self.h2_3 = 256
        
        self.h3_1 = 256
        
        self.head1 = nn.Sequential(
            nn.BatchNorm1d(num_features1),
            nn.Dropout(0.3),
            nn.Linear(num_features1, self.h1_1),
            nn.LeakyReLU(),
            
            nn.BatchNorm1d(self.h1_1),
            nn.Linear(self.h1_1, self.h1_2),
            nn.LeakyReLU(),
        )
        
        self.head2 = nn.Sequential(
            nn.BatchNorm1d(self.h2_1),
            nn.Dropout(0.30),
            nn.Linear(self.h2_1, self.h2_2),
            nn.ReLU(),
            
            nn.BatchNorm1d(self.h2_2),
            nn.Linear(self.h2_2, self.h2_2),
            nn.ELU(),            
            
            nn.BatchNorm1d(self.h2_2),
            nn.Linear(self.h2_2, self.h2_3),
            nn.ReLU(),  
            
            nn.BatchNorm1d(self.h2_3),
            nn.Linear(self.h2_3, self.h2_3),
            nn.ELU(),            
        )
        self.head3 = nn.Sequential(
            nn.BatchNorm1d(self.h3_1),
            nn.Linear(self.h3_1, self.h3_1),
            nn.LeakyReLU(),
            
            nn.BatchNorm1d(self.h3_1),
            nn.Linear(self.h3_1, num_targets),
            nn.LeakyReLU(),
            
            nn.BatchNorm1d(num_targets),
            nn.Linear(num_targets, num_targets),
        )

    
    def forward(self, input1, input2):
        input3 = self.head1(input1)
        concat = torch.cat((input3, input2), dim=1)
        input4 = self.head2(concat)
        avg = torch.div(torch.add(input3, input4), 2)
        
        out = self.head3(avg)
        
        return out
    """

class Model(nn.Module):
    def __init__(self, num_features1, num_features2, num_targets):
        super(Model, self).__init__()
        self.h1_1 = 1024
        self.h1_2 = 512  #
        
        self.h2_1 = num_features2+self.h1_2
        self.h2_2 = 1024
        self.h2_3 = 512  #
        
        self.h3_1 = 512
        
        self.head1 = nn.Sequential(
            nn.BatchNorm1d(num_features1),
            nn.Linear(num_features1, self.h1_1),
            nn.LeakyReLU(),
            
            nn.BatchNorm1d(self.h1_1),
            nn.Dropout(0.45),
            nn.Linear(self.h1_1, self.h1_2),
            nn.LeakyReLU(),
        )
        
        self.head2 = nn.Sequential(
            nn.BatchNorm1d(self.h2_1),
            nn.Linear(self.h2_1, self.h2_2),
            nn.ReLU(),
            
            nn.BatchNorm1d(self.h2_2),
            nn.Dropout(0.45),
            nn.Linear(self.h2_2, self.h2_2),
            nn.ELU(),            
            
            nn.BatchNorm1d(self.h2_2),
            nn.Dropout(0.45),
            nn.Linear(self.h2_2, self.h2_3),
            nn.ReLU(),  
            
            nn.BatchNorm1d(self.h2_3),
            nn.Dropout(0.45),
            nn.Linear(self.h2_3, self.h2_3),
            nn.ELU(),            
        )
        self.head3 = nn.Sequential(
            nn.BatchNorm1d(self.h3_1),
            nn.Dropout(0.45),
            nn.Linear(self.h3_1, self.h3_1),
            nn.LeakyReLU(),
            
            nn.BatchNorm1d(self.h3_1),
            nn.Linear(self.h3_1, self.h3_1),
            nn.LeakyReLU(),
            
            nn.BatchNorm1d(self.h3_1),
            nn.Linear(self.h3_1, num_targets),
        )

    
    def forward(self, input1, input2):
        input3 = self.head1(input1)
        concat = torch.cat((input3, input2), dim=1)
        input4 = self.head2(concat)
        avg = torch.div(torch.add(input3, input4), 2)
        
        out = self.head3(avg)
        
        return out
        


## run train

In [11]:
def run_training(model, trainloader, validloader, epoch_, optimizer, scheduler, loss_fn, loss_tr, early_stopping_steps, verbose, device, fold, seed):
    
    early_step = 0
    best_loss = np.inf
    best_epoch = 0
    
    start = time.time()
    t = time.time() - start
    for epoch in range(epoch_):
        train_loss = train_fn(model, optimizer, scheduler, loss_tr, trainloader, device)
        valid_loss = valid_fn(model, loss_fn, validloader, device)
        # if ReduceLROnPlateau
        #scheduler.step(valid_loss)
        if epoch % verbose==0:
            t = time.time() - start
            print(f"EPOCH: {epoch}, train_loss: {train_loss}, valid_loss: {valid_loss}, time: {t}")
        
        if valid_loss < best_loss:
            best_loss = valid_loss
            torch.save(model.state_dict(), 'resnet_weights/{}_{}.pt'.format(seed, fold))
            #torch.save(model, 'dnn_weights/{}_{}.pt'.format(seed, fold))
            early_step = 0
            best_epoch = epoch
        
        elif early_stopping_steps != 0:
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                t = time.time() - start
                print(f"early stopping in iteration {epoch},  : best itaration is {best_epoch}, valid loss is {best_loss}, time: {t}")
                return model
    t = time.time() - start 
    print(f"training until max epoch {epoch_},  : best itaration is {best_epoch}, valid loss is {best_loss}, time: {t}")
    return model
            
    
def predict(model, testloader, device):
    model.to(device)
    predictions = inference_fn(model, testloader, device)
    
    return predictions

In [12]:
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

## Training by Fold

In [13]:
BATCH_SIZE = 128
DEVICE = ('cuda:2' if torch.cuda.is_available() else 'cpu')
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
EPOCHS = 25
EARLY_STOPPING_STEPS = 10

train_preds = np.zeros((X.shape[0], y_nonv.shape[1]))
preds = np.zeros((test_df.shape[0], y_nonv.shape[1]))
imps = []
imp_cols = []
folds = []
test_cv_preds = []

for seed in seeds:
    seed_everything(seed)
    K = 5
    kf = MultilabelStratifiedKFold(n_splits=K, random_state=seed, shuffle=True)
    train_pred = np.zeros(train_preds.shape)
    
    
    ###############################################################################################
    # LOAD LIBRARIES
    targets = SCORED_MOAS.copy()

    # LOCATE DRUGS
    vc = scored["drug_id"].value_counts()
    vc1 = vc.loc[vc<=18].index.sort_values()
    vc2 = vc.loc[vc>18].index.sort_values()

    # STRATIFY DRUGS 18X OR LESS
    dct1 = {}; dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
    tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.index[idxV].values} # drug id がどのフォールドに属すか格納
        dct1.update(dd)

    # STRATIFY DRUGS MORE THAN 18X
    skf = MultilabelStratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
    tmp = scored.loc[scored["drug_id"].isin(vc2)].reset_index(drop=True)
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp["sig_id"][idxV].values}
        dct2.update(dd)

    # ASSIGN K
    scored['fold'] = scored.drug_id.map(dct1)
    scored.loc[scored["fold"].isna(),'fold'] = scored.loc[scored["fold"].isna(),'sig_id'].map(dct2)
    scored["fold"] = scored["fold"].astype('int8')
    ###############################################################################################

    #for fold, (train_index, valid_index) in enumerate(kf.split(X, y_nonv)):    
    for fold in range(K):
        train_index = scored[scored["fold"] != fold].index.to_list()
        valid_index = scored[scored["fold"] == fold].index.to_list()
        print("======================== fold {} ========================".format(fold+1))
        folds.append(train_index)
                
        # split data
        train_X = X.iloc[train_index]
        train_y = y_nonv[train_index]
        valid_X = X.iloc[valid_index]
        valid_y = y_nonv[valid_index]
        test_X = (test_df.drop("sig_id", axis=1))
        pub_test_X = (pub_test_df.drop("sig_id", axis=1))
        
        

        # scaler
        print(SCALE)
        scale_cols = BIOS_+PRODS
        scaler = make_scaler(SCALE, seed).fit(train_X.append(pub_test_X)[scale_cols])
        for df in [train_X, valid_X, test_X, pub_test_X]:
            df[scale_cols] = scaler.transform(df[scale_cols])
            
            
        print("PCA")
        #PCA
        n_decom_g = 80
        n_decom_c = 10
        decom_g_cols = [f"pca_g-{i}" for i in range(n_decom_g)]
        decom_c_cols = [f"pca_c-{i}" for i in range(n_decom_c)]
        
        pca_g = PCA(n_components = n_decom_g, random_state = seed).fit(train_X.append(pub_test_X)[GENES_])
        pca_c = PCA(n_components = n_decom_c, random_state = seed).fit(train_X.append(pub_test_X)[CELLS_])
        
        for df in [train_X, valid_X, test_X, pub_test_X]:
            df[decom_g_cols] = pca_g.transform(df[GENES_])
            df[decom_c_cols] = pca_c.transform(df[CELLS_])
            #df.drop(GENES_+CELLS_, axis=1, inplace=True) 
            
        
        print(train_X.shape[0])
        # prepare data for training
        train_X1 = train_X.values
        train_X2 = train_X.values
        valid_X1 = valid_X.values
        valid_X2 = valid_X.values
        test_X1 = test_X.values
        test_X2 = test_X.values
        print(train_X1.shape[1])
        print(train_X2.shape[1])
        
        
        
        # ================================model training===========================
        train_dataset = MoAResNetDataset(train_X1, train_X2, train_y)
        valid_dataset = MoAResNetDataset(valid_X1, valid_X2, valid_y)
        test_dataset = TestResNetDataset(test_X1, test_X2)
        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
        validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
        testloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

        model = Model(
            num_features1=train_X1.shape[1],
            num_features2=train_X2.shape[1],
            num_targets=train_y.shape[1],
        )

        model.to(DEVICE)
        
        
        optimizer = torch.optim.Adam( model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY,)
        
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer,pct_start=0.1, div_factor=1e3, max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader) )
        #scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,mode = "min", patience = 3, min_lr = 1e-5, factor = 0.1, eps=1e-5)
        
        loss_fn = nn.BCEWithLogitsLoss()
        loss_tr = SmoothBCEwLogits(smoothing=1e-3)
        
        
        # train
        model = run_training(
            model=model,
            trainloader=trainloader,
            validloader=validloader,
            epoch_=EPOCHS,
            optimizer=optimizer,
            scheduler=scheduler,
            loss_fn=loss_fn,
            loss_tr=loss_tr,
            early_stopping_steps=EARLY_STOPPING_STEPS,
            device=DEVICE,
            verbose=5,
            fold=fold,
            seed=seed,)
        #model = torch.load('dnn_weights/{}_{}.pt'.format(seed, fold))
        model.load_state_dict(torch.load('resnet_weights/{}_{}.pt'.format(seed, fold)))
        
        #valid predict
        val_preds = predict(
            model=model,
            testloader=validloader,
            device=DEVICE,)
        
        #test predict
        test_preds = predict(
            model=model,
            testloader=testloader,
            device=DEVICE)
        
        # ================================model training===========================

        train_pred[valid_index] +=  val_preds
        
        preds += test_preds / (K*len(seeds))
        
        #name = "{}_{}".format(seed, fold)
        #model.save_model("tabnet_weights_inf/"+name)
        #imps.append(model.feature_importances_)

    print("seed {} , cv score : {}".format(seed, metric(y_nonv, train_pred)))
    train_preds += train_pred/len(seeds)

print("cv score : {}".format(metric(y_nonv, train_preds)))

======================== fold 1 ========================
quantile
PCA
17584
915
915
EPOCH: 0, train_loss: 0.4694377307229451, valid_loss: 0.021487450546451976, time: 5.856710910797119
EPOCH: 5, train_loss: 0.021223246534592913, valid_loss: 0.01839931370424373, time: 24.223227739334106
EPOCH: 10, train_loss: 0.021200454297618274, valid_loss: 0.01835889361266579, time: 41.74117588996887
EPOCH: 15, train_loss: 0.02046806350295996, valid_loss: 0.017957312054932117, time: 59.15132522583008
EPOCH: 20, train_loss: 0.019242459962511584, valid_loss: 0.017561373274241177, time: 76.91962194442749
training until max epoch 25,  : best itaration is 21, valid loss is 0.017340123653411865, time: 90.86381435394287
======================== fold 2 ========================
quantile
PCA
17565
915
915
EPOCH: 0, train_loss: 0.4692256659486868, valid_loss: 0.02058689266975437, time: 3.302098274230957
EPOCH: 5, train_loss: 0.021403426331651473, valid_loss: 0.017949399831039565, time: 21.009947299957275
EPOCH: 

EPOCH: 15, train_loss: 0.020641604152909162, valid_loss: 0.017445248332532012, time: 55.13194251060486
EPOCH: 20, train_loss: 0.01942501819427431, valid_loss: 0.01704418325029752, time: 72.9765989780426
training until max epoch 25,  : best itaration is 22, valid loss is 0.016940455911133218, time: 87.92011094093323
======================== fold 3 ========================
quantile
PCA
17595
915
915
EPOCH: 0, train_loss: 0.4694569824929655, valid_loss: 0.021489183232188225, time: 3.3637495040893555
EPOCH: 5, train_loss: 0.0213659369543086, valid_loss: 0.018591551748769624, time: 21.21135187149048
EPOCH: 10, train_loss: 0.021316560352370686, valid_loss: 0.01851958403629916, time: 39.44395923614502
EPOCH: 15, train_loss: 0.020708528694010127, valid_loss: 0.01780521007520812, time: 57.203413248062134
EPOCH: 20, train_loss: 0.019376850802532947, valid_loss: 0.017000470363668033, time: 75.02630710601807
training until max epoch 25,  : best itaration is 24, valid loss is 0.01685239768454007, t

PCA
17537
915
915
EPOCH: 0, train_loss: 0.47038447586343674, valid_loss: 0.020970758742519788, time: 3.315481185913086
EPOCH: 5, train_loss: 0.021296065912520798, valid_loss: 0.018257283685462814, time: 21.65800952911377
EPOCH: 10, train_loss: 0.021222781743446405, valid_loss: 0.01825995503791741, time: 39.029476165771484
EPOCH: 15, train_loss: 0.020570263552078364, valid_loss: 0.01780661360493728, time: 57.07944917678833
EPOCH: 20, train_loss: 0.019320333063819983, valid_loss: 0.01727658249437809, time: 75.51666569709778
training until max epoch 25,  : best itaration is 24, valid loss is 0.017168763226696422, time: 90.55651021003723
======================== fold 5 ========================
quantile
PCA
17567
915
915
EPOCH: 0, train_loss: 0.47028640336798927, valid_loss: 0.02151911681784051, time: 3.3170018196105957
EPOCH: 5, train_loss: 0.021289461135973027, valid_loss: 0.01863963593329702, time: 22.045066595077515
EPOCH: 10, train_loss: 0.021260988293555532, valid_loss: 0.018322679001

EPOCH: 20, train_loss: 0.019373996607849702, valid_loss: 0.01738066521606275, time: 73.2342803478241
training until max epoch 25,  : best itaration is 23, valid loss is 0.01723500868039472, time: 87.19083499908447
seed 13 , cv score : 0.0173115250690438
cv score : 0.017054597698718785


In [14]:
train_preds2 = np.zeros((TR_SIZE,  y.shape[1]))
train_preds2[train_nonvehicle_index] = train_preds


preds2 = np.zeros((TE_SIZE, y.shape[1]))
preds2[test_nonvehicle_index] = preds

print("cv score : {}".format(metric(y, train_preds2)))

cv score : 0.01571824600199386


In [15]:
sub_df = pd.read_csv("../../../Data/Raw/sample_submission.csv")
#sub_df = pd.read_csv("../input/lish-moa/sample_submission.csv")
cols = [col for col in sub_df.columns if col != "sig_id"]
sub_df[cols] = preds2
sub_df.to_csv("submission.csv", index=False)

In [16]:
train_sub = pd.read_csv("../../../Data/Raw/train_targets_scored.csv")
cols = [col for col in train_sub.columns if col != "sig_id"]
train_sub[cols] = train_preds2
train_sub.to_csv("train_preds.csv", index=False)

In [17]:
!zip resnet_weights.zip resnet_weights/*

updating: resnet_weights/10_0.pt (deflated 3%)
updating: resnet_weights/10_1.pt (deflated 3%)
updating: resnet_weights/10_2.pt (deflated 3%)
updating: resnet_weights/10_3.pt (deflated 3%)
updating: resnet_weights/10_4.pt (deflated 3%)
updating: resnet_weights/11_0.pt (deflated 3%)
updating: resnet_weights/11_1.pt (deflated 3%)
updating: resnet_weights/11_2.pt (deflated 3%)
updating: resnet_weights/11_3.pt (deflated 3%)
updating: resnet_weights/11_4.pt (deflated 3%)
updating: resnet_weights/12_0.pt (deflated 3%)
updating: resnet_weights/12_1.pt (deflated 3%)
updating: resnet_weights/12_2.pt (deflated 3%)
updating: resnet_weights/12_3.pt (deflated 3%)
updating: resnet_weights/12_4.pt (deflated 3%)
updating: resnet_weights/13_0.pt (deflated 3%)
updating: resnet_weights/13_1.pt (deflated 3%)
updating: resnet_weights/13_2.pt (deflated 3%)
updating: resnet_weights/13_3.pt (deflated 3%)
updating: resnet_weights/13_4.pt (deflated 3%)
updating: resnet_weights/7_0.pt (deflated 3%)
updating: resn